In [1]:
import os
import sys
import tensorflow                             as tf
import numpy                                  as np
import pandas                                 as pd
import matplotlib.pyplot                      as plt

In [2]:
WORKSPACE_PATH    = os.getenv('WORKSPACE_PATH')  
TVec              = np.array([1500.0, 5000.0, 10000.0, 15000.0, 20000.0])
NPerGroup         = 3

Molecule          = 'O2'
PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp'
PathTo3AtomsFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/AmalInel_Sampled/'
PathToGroups      = WORKSPACE_PATH + '/Air_Database/Run_0D/database/grouping/O3_UMN/O2/LevelsMap_InelAmal45.csv'

# Molecule          = 'N2'
# PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp'
# PathTo3AtomsFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/AmalInel_Sampled/'
# PathToGroups      = WORKSPACE_PATH + '/Air_Database/Run_0D/database/grouping/N4_NASA/N2/LevelsMap_InelAmal61.csv'

In [3]:
LevelToGroupDF    = pd.read_csv(PathToGroups)
LevelToGroup      = LevelToGroupDF.to_numpy(int)[:,1] - 1
NGroup            = np.amax(LevelToGroup) + 1

In [4]:
### Reading Levels Data
LevelsData           = pd.read_csv(PathToDiatPot, delim_whitespace=True, skiprows=15, header=None)
LevelsData.columns   = ['vqn','jqn','EInt','egam','rMin','rMax','VMin','VMax','Tau','ri','ro']    


### Shifting Energies so that Zero is the Min of Diatomic Potential at J=0
LevelsData['EInt']   = (LevelsData['EInt'].to_numpy() -  np.amin(LevelsData['VMin'].to_numpy())) * 27.211399
LevelsData['VMax']   = (LevelsData['VMax'].to_numpy() -  np.amin(LevelsData['VMin'].to_numpy())) * 27.211399
LevelsData['VMin']   = (LevelsData['VMin'].to_numpy() -  np.amin(LevelsData['VMin'].to_numpy())) * 27.211399


### Groups
LevelsData['Group']  = LevelToGroup + 1


LevelsData.to_csv('./'+Molecule+'Groups_Amals'+str(NPerGroup)+'.csv', index=False)

In [5]:
for T in TVec:
    
    Sampled       = pd.read_csv(PathTo3AtomsFldr+str(Molecule)+'_Sampled_Inel_'+str(int(T))+'K.csv').to_numpy(int)[:,0] - 1
    
    LevelsDataNow = LevelsData.iloc[Sampled]
    
    LevelsDataNow.to_csv('./'+Molecule+'SampledGroups_Amals'+str(NPerGroup)+'_'+str(int(T))+'.csv', index=False)

In [6]:
LevelsDataOld = LevelsData.iloc[[4579,4580]]

LevelsDataOld.to_csv('./'+Molecule+'SampledLevels_Missing.csv', index=False)